# Generates Table of detection probabilities and key metrics for the CPP Targets Wiki

### Imports

In [ ]:
# Autoreload extension
%load_ext autoreload
%autoreload 2

In [ ]:
import os, numpy as np, pandas as pd
from astropy.time import Time
from astropy.table import Table
from decimal import Decimal
from astropy import units as u, constants as c
import matplotlib.pyplot as plt
from roman_pointing.roman_pointing import calcRomanAngles, getL2Positions
from astroquery.simbad import Simbad
from astropy.coordinates import (
    SkyCoord,
    Distance,
    BarycentricMeanEcliptic,
)
from roman_table import *

### Helper Functions

In [ ]:
def get_GB_sunang(ts):
    # ts must be astropy.Time object
    simbad = Simbad()
    simbad.add_votable_fields("pmra", "pmdec", "plx_value", "rvz_radvel")
    res = simbad.query_object("Sagittarius A*")
    gb = SkyCoord(
        res["ra"].value.data[0],
        res["dec"].value.data[0],
        unit=(res["ra"].unit, res["dec"].unit),
        frame="icrs",
        distance=Distance(8 * u.kpc),
        pm_ra_cosdec=0 * res["pmra"].unit,
        pm_dec=0 * res["pmdec"].unit,
        radial_velocity=0 * res["rvz_radvel"].unit,
        equinox="J2000",
        obstime="J2000",
    ).transform_to(BarycentricMeanEcliptic)
    sun_ang_ref, _, _, _ = calcRomanAngles(
        gb, ts, getL2Positions(ts)
    )
    return sun_ang_ref

def get_targ_sunang(star,ts):
    # ts must be astropy.Time object
    simbad_name = " ".join(star.split("_"))
    simbad = Simbad()
    simbad.add_votable_fields("pmra", "pmdec", "plx_value", "rvz_radvel")
    res = simbad.query_object(simbad_name)
    target = SkyCoord(
        res["ra"].value.data[0],
        res["dec"].value.data[0],
        unit=(res["ra"].unit, res["dec"].unit),
        frame="icrs",
        distance=Distance(parallax=res["plx_value"].value.data[0] * res["plx_value"].unit),
        pm_ra_cosdec=res["pmra"].value.data[0] * res["pmra"].unit,
        pm_dec=res["pmdec"].value.data[0] * res["pmdec"].unit,
        radial_velocity=res["rvz_radvel"].value.data[0] * res["rvz_radvel"].unit,
        equinox="J2000",
        obstime="J2000",
    ).transform_to(BarycentricMeanEcliptic)

    sun_ang_targ, _, _, _ = calcRomanAngles(
        target, ts, getL2Positions(ts)
    )
    return sun_ang_targ

### Main

In [ ]:
# Load contrast curve
concurve_fpath = 'concurves/concurve_optimistic.csv'
concurve_df = pd.read_csv(concurve_fpath)
concurve_df['D'] = 2.4
concurve_df['sep_mas'] = concurve_df.loD * concurve_df['lambda'] * 1e-9 / concurve_df['D'] * u.radian.to(u.mas)
display(concurve_df)
concurve = np.array([
    concurve_df.sep_mas.values,
    concurve_df.contr.values])

In [ ]:
planets = list(orbit_params.keys())
start_date, end_date = "2027-01-01", "2028-06-30"
plot=True
orbit_props_dir = '/Users/sbogat/Documents/01_Research/exoplanner_workspace/output/orbit_props'
bands = [1,3,4]

# Temporary while we wait for albedos from Dmitry
# Band wls from: https://roman.ipac.caltech.edu/docs/Coronagraph_Technical_Information.updated.pdf
# Albedos from: https://ui.adsabs.harvard.edu/abs/1998Icar..133..134K/abstract
albedo_dict = { 
    1 : 0.465,
    3 : 0.4,
    4 : 0.35,
}
albedo_std = 0.1

semesters = [
    ("2027-01-01", "2027-06-30"),
    ("2027-07-01", "2027-12-31"),
    ("2028-01-01", "2028-06-30"),
]

summary_dict = {
    'planet' : []
}

for sem in range(len(semesters)):
    sem += 1
    for band in bands:
        summary_dict[f'peak_pdet_sem{sem}_band{band}'] = []
        summary_dict[f'peak_date_sem{sem}_band{band}'] = []
        summary_dict[f'mean_pdet_sem{sem}_band{band}'] = []
        summary_dict[f'sep_range_sem{sem}_band{band}'] = []
        summary_dict[f'fc_range_sem{sem}_band{band}'] = []
        summary_dict[f'mag_range_sem{sem}_band{band}'] = []

for p,planet in enumerate(planets):
    summary_dict['planet'].append(planet)
        
    for band in bands:

        fname_in = f"{planet}_{start_date}_to_{end_date}_RVOnly"
        fname_out = fname_in + f'_Band{band}'
        # Load saved point cloud
        print(f'Planet index {p}/{len(planets)-1}: {planet}')
        point_cloud_fname = f"{fname_in}_PointCloud.pkl"
        point_cloud = load_point_cloud(planet,
                        i_dir=orbit_props_dir,
                        fname=point_cloud_fname
                        )
        
        # Calculate flux contrast (TODO: Update to use albedos/phasecurve from dmitry)
        print('Calculating flux contrast & detectability...')
        albedos = np.random.normal(albedo_dict[band],albedo_std,size=point_cloud['sep_mas'].shape)
        phase_angle_rad = point_cloud['phase_angle_deg'] * np.pi / 180.
        point_cloud['lambert_phase'] = (np.sin(phase_angle_rad)+(np.pi-phase_angle_rad)*np.cos(phase_angle_rad))/np.pi
        point_cloud['phi_x_a'] = point_cloud['lambert_phase'] * albedos
        
        point_cloud['flux_contrast'] = point_cloud['phi_x_a'] * (point_cloud['r_pl_rjup']*u.R_jup.to(u.AU) / point_cloud['orbital_radius_au'])**2

        point_cloud['is_detectable'] = is_detectable(point_cloud['sep_mas'],
                                                    point_cloud['flux_contrast'],
                                                    concurve)
        
        point_cloud['detection_probability'] = weighted_mean(point_cloud['is_detectable'],
                                                        weights=point_cloud['ln_likelihood'])
        
        # Calculate Observation windows
        print('Getting target observation windows')
        times = Time(point_cloud['epoch_mjd'][:,0],format='mjd')
        sun_ang_ref = get_GB_sunang(times)
        point_cloud['GB_not_observable'] = ~((sun_ang_ref.to_value(u.deg) > 54) & (sun_ang_ref.to_value(u.deg) < 126))
        star = orbit_params[planet]['star']
        sun_ang_targ = get_targ_sunang(star,times)
        point_cloud['targ_observable'] = (sun_ang_targ.to_value(u.deg) > 54) & (sun_ang_targ.to_value(u.deg) < 126)    

        # Update summary csv
        print('Updating summary csv.')
        csv_data = gen_summary_csv(
            planet,
            point_cloud,
            orbit_props_dir,
            fname_out+'_withDetProb')   
        
        # Calculate stats for each semester
        for s,semester in enumerate(semesters):

            # Get the sections of the table in just the semester of interest
            snum=s+1
            s_start, s_end = Time(list(semester))
            dates_iso = Time(list(csv_data.date_iso.values))
            sem_df = csv_data.loc[(dates_iso>s_start) & (dates_iso<s_end)]

            peak_pdet = str('%.2f'%(np.max(sem_df.det_probability))).lstrip('-')
            peak_date = sem_df.date_iso.iloc[np.argmax(sem_df.det_probability)][:10]
            mean_pdet = str('%.2f'%(np.mean(sem_df.det_probability))).lstrip('-')
            sep_range = f'{np.round(np.min(sem_df.separation_mas_16th),0)} - {np.round(np.max(sem_df.separation_mas_84th),0)}'
            fc_range = '%.2E'%Decimal(np.min(sem_df.flux_contrast_16th)) + ' - ' + '%.2E'%Decimal(np.max(sem_df.flux_contrast_84th))

            print(f'Peak det prob {peak_pdet} on {peak_date}')

            summary_dict[f'peak_pdet_sem{snum}_band{band}'].append(peak_pdet)
            summary_dict[f'peak_date_sem{snum}_band{band}'].append(peak_date)
            summary_dict[f'mean_pdet_sem{snum}_band{band}'].append(mean_pdet)
            summary_dict[f'sep_range_sem{snum}_band{band}'].append(sep_range)
            summary_dict[f'fc_range_sem{snum}_band{band}'].append(fc_range)
            summary_dict[f'mag_range_sem{snum}_band{band}'].append(None)

        print('Plotting...')
        plot_outfpath = os.path.join(orbit_props_dir,fname_out)
        plot_orbital_parameters(planet,csv_data,plot_outfpath,
                                    override_lan=0.,
                                    start_date=start_date,end_date=end_date,
                                    figsize=None,fig_ext='png',
                                    show_plots=True,
                                    band=band)

In [ ]:
# Display detection probabilities in format expected by wiki pages
planet_order = [
    'ups And d',
    'eps Eri b',
    '47 UMa b',
    '47 UMa c',
    '47 UMa d',
    'HD 154345 b',
    'HD 190360 b',
    'HD 217107 c',
    'HD 192310 c',
    '55 Cnc d',
    'HD 160691 c',
    'pi Men b',
    'HD 134987 c',
    'HD 87883 b',
    'HD 114783 c',
    '14 Her b',
    '14 Her c',
]

# new_index = ['_'.join(planet.split(' ')) for planet in planet_order]

alltargs_df = pd.DataFrame(summary_dict)
alltargs_df.sort_values('peak_pdet_sem1_band1',ascending=False,inplace=True)
alltargs_table = Table.from_pandas(alltargs_df)
display(alltargs_table['planet',
                       'peak_pdet_sem1_band1','peak_pdet_sem2_band1','peak_pdet_sem3_band1'])

def extract_single_targ_pdet_summary(alltargs_df,planet_index,band):
    columns = ['mean_pdet','peak_pdet_window','separation_range','flux_contrast_range','mag_range']
    singletarg_dict = {}
    for s in range(len(semesters)):
        s += 1
        mean_pdet = alltargs_df.loc[planet_index,f'mean_pdet_sem{s}_band{band}'].lstrip('-')
        peak_pdetanddate = f"{alltargs_df.loc[planet_index,f'peak_pdet_sem{s}_band{band}']}, {alltargs_df.loc[planet_index,f'peak_date_sem{s}_band{band}']}".lstrip('-')

        sep_range = alltargs_df.loc[planet_index,f'sep_range_sem{s}_band{band}']
        fc_range = alltargs_df.loc[planet_index,f'fc_range_sem{s}_band{band}']
        mag_range = alltargs_df.loc[planet_index,f'mag_range_sem{s}_band{band}']
        singletarg_dict[s] = [mean_pdet,peak_pdetanddate,sep_range,fc_range,mag_range]
    singletarg_df = pd.DataFrame.from_dict(singletarg_dict,orient='index',columns=columns)
    
    return singletarg_df

temp = alltargs_df.set_index('planet')

for p in temp.index:
    print(p)
    p = '_'.join(p.split(' '))
    if p in temp.index:
        for band in bands:
            print(f'Band {band}')
            singletarg_df = extract_single_targ_pdet_summary(temp,p,band)
            singletargs_table = Table.from_pandas(singletarg_df)
            display(singletargs_table)


In [ ]:
new_index